# Test that file handler can download things and use the downloaded copy before downloading again

In [2]:
import sys
sys.path.append('../')

import helpers.fileHandler as fh
import helpers.esgfClient as esgfClient

In [22]:
esgfClient.esgfDownloader('MPI-ESM1-2-HR', 'pr_Amon', 'piControl', 'r1i1p1f1')

CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.piControl.r1i1p1f1.Amon.pr.gn.v20190710|esgf-data1.llnl.gov downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.piControl.r1i1p1f1.Amon.pr.gn.v20190710|esgf-data3.diasjp.net downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.piControl.r1i1p1f1.Amon.pr.gn.v20190710|esgf3.dkrz.de downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.piControl.r1i1p1f1.Amon.pr.gn.v20190710|esgf.nci.org.au downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.piControl.r1i1p1f1.Amon.pr.gn.v20190710|esgf-data3.ceda.ac.uk downloading


True

In [23]:
esgfClient.esgfDownloader('MPI-ESM1-2-LR', 'pr_Amon', 'piControl', 'r1i1p1f1')

CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Amon.pr.gn.v20190710|esgf-data1.llnl.gov downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Amon.pr.gn.v20190710|esgf-data3.diasjp.net downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Amon.pr.gn.v20190710|esgf3.dkrz.de downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Amon.pr.gn.v20190710|esgf.nci.org.au downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Amon.pr.gn.v20190710|esgf-data3.ceda.ac.uk downloading


True

In [24]:
esgfClient.esgfDownloader('MPI-ESM1-2-LR', 'tas_Amon', 'piControl', 'r1i1p1f1')

CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Amon.tas.gn.v20190710|esgf-data1.llnl.gov downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Amon.tas.gn.v20190710|esgf-data3.diasjp.net downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Amon.tas.gn.v20190710|esgf3.dkrz.de downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Amon.tas.gn.v20190710|esgf.nci.org.au downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.piControl.r1i1p1f1.Amon.tas.gn.v20190710|esgf-data3.ceda.ac.uk downloading


True

In [25]:
esgfClient.esgfDownloader('MRI-ESM2-0', 'tas_Amon', 'piControl', 'r1i1p1f1')

CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Amon.tas.gn.v20190222|aims3.llnl.gov downloading
CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Amon.tas.gn.v20190222|esgf-data2.diasjp.net downloading
CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Amon.tas.gn.v20190222|esgf3.dkrz.de downloading
CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Amon.tas.gn.v20190222|esgf.nci.org.au downloading
CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Amon.tas.gn.v20190222|esgf-data3.ceda.ac.uk downloading


True

In [15]:
esgfClient.esgfDownloader('GISS-E2-1-G', 'tas_Amon', 'ssp126', 'r1i1p1f2')

CMIP6.ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp126.r1i1p1f2.Amon.tas.gn.v20200115|esgf-data1.llnl.gov downloading
CMIP6.ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp126.r1i1p1f2.Amon.tas.gn.v20200115|esgf-data3.diasjp.net downloading
CMIP6.ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp126.r1i1p1f2.Amon.tas.gn.v20200115|dpesgf03.nccs.nasa.gov downloading
CMIP6.ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp126.r1i1p1f2.Amon.tas.gn.v20200115|esgf.nci.org.au downloading
CMIP6.ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp126.r1i1p1f2.Amon.tas.gn.v20200115|esgf-data3.ceda.ac.uk downloading


True

In [16]:
esgfClient.esgfDownloader('HadGEM3-GC31-LL','pr_Amon', 'piControl','r1i1p1f1')

CMIP6.CMIP.MOHC.HadGEM3-GC31-LL.piControl.r1i1p1f1.Amon.pr.gn.v20190628|aims3.llnl.gov downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-LL.piControl.r1i1p1f1.Amon.pr.gn.v20190628|esgf-data3.diasjp.net downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-LL.piControl.r1i1p1f1.Amon.pr.gn.v20190628|esgf3.dkrz.de downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-LL.piControl.r1i1p1f1.Amon.pr.gn.v20190628|esgf.nci.org.au downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-LL.piControl.r1i1p1f1.Amon.pr.gn.v20190628|esgf-data3.ceda.ac.uk downloading


True

In [17]:
esgfClient.esgfDownloader('HadGEM3-GC31-LL','tas_Amon', 'piControl','r1i1p1f1')

CMIP6.CMIP.MOHC.HadGEM3-GC31-LL.piControl.r1i1p1f1.Amon.tas.gn.v20190628|aims3.llnl.gov downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-LL.piControl.r1i1p1f1.Amon.tas.gn.v20190628|esgf-data3.diasjp.net downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-LL.piControl.r1i1p1f1.Amon.tas.gn.v20190628|esgf3.dkrz.de downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-LL.piControl.r1i1p1f1.Amon.tas.gn.v20190628|esgf.nci.org.au downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-LL.piControl.r1i1p1f1.Amon.tas.gn.v20190628|esgf-data3.ceda.ac.uk downloading


True

In [18]:
esgfClient.esgfDownloader('HadGEM3-GC31-MM','pr_Amon', 'piControl','r1i1p1f1')

CMIP6.CMIP.MOHC.HadGEM3-GC31-MM.piControl.r1i1p1f1.Amon.pr.gn.v20190920|esgf-data1.llnl.gov downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-MM.piControl.r1i1p1f1.Amon.pr.gn.v20190920|esgf-data3.diasjp.net downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-MM.piControl.r1i1p1f1.Amon.pr.gn.v20190920|esgf3.dkrz.de downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-MM.piControl.r1i1p1f1.Amon.pr.gn.v20190920|esgf.nci.org.au downloading
CMIP6.CMIP.MOHC.HadGEM3-GC31-MM.piControl.r1i1p1f1.Amon.pr.gn.v20190920|esgf-data3.ceda.ac.uk downloading


True

In [19]:
esgfClient.esgfDownloader('INM-CM4-8','pr_Amon', 'piControl','r1i1p1f1')

CMIP6.CMIP.INM.INM-CM4-8.piControl.r1i1p1f1.Amon.pr.gr1.v20190605|esgf-data1.llnl.gov downloading
CMIP6.CMIP.INM.INM-CM4-8.piControl.r1i1p1f1.Amon.pr.gr1.v20190605|esgf-data3.diasjp.net downloading
CMIP6.CMIP.INM.INM-CM4-8.piControl.r1i1p1f1.Amon.pr.gr1.v20190605|esgf3.dkrz.de downloading
CMIP6.CMIP.INM.INM-CM4-8.piControl.r1i1p1f1.Amon.pr.gr1.v20190605|esgf.nci.org.au downloading
CMIP6.CMIP.INM.INM-CM4-8.piControl.r1i1p1f1.Amon.pr.gr1.v20190605|esgf-data3.ceda.ac.uk downloading


True

In [6]:
esgfClient.esgfDownloader('MPI-ESM1-2-HR','psl_Amon', 'piControl','r1i1p1f1')

CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.piControl.r1i1p1f1.Amon.psl.gn.v20190710|esgf-data1.llnl.gov downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.piControl.r1i1p1f1.Amon.psl.gn.v20190710|esgf-data3.diasjp.net downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.piControl.r1i1p1f1.Amon.psl.gn.v20190710|esgf3.dkrz.de downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.piControl.r1i1p1f1.Amon.psl.gn.v20190710|esgf.nci.org.au downloading
CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.piControl.r1i1p1f1.Amon.psl.gn.v20190710|esgf-data3.ceda.ac.uk downloading


True

In [7]:
esgfClient.esgfDownloader('EC-Earth3-Veg-LR','ts_Amon', 'piControl','r1i1p1f1')

CMIP6.CMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.piControl.r1i1p1f1.Amon.ts.gr.v20200213|esgf-data3.diasjp.net downloading
CMIP6.CMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.piControl.r1i1p1f1.Amon.ts.gr.v20200213|esgf-data1.llnl.gov downloading
CMIP6.CMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.piControl.r1i1p1f1.Amon.ts.gr.v20200213|esg-dn1.nsc.liu.se downloading
CMIP6.CMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.piControl.r1i1p1f1.Amon.ts.gr.v20200213|esgf-data3.ceda.ac.uk downloading


True

In [3]:
esgfClient.esgfDownloader('EC-Earth3-CC','tas_Amon', 'piControl','r1i1p1f1')

CMIP6.CMIP.EC-Earth-Consortium.EC-Earth3-CC.piControl.r1i1p1f1.Amon.tas.gr.v20210330|esgf-data3.diasjp.net downloading
CMIP6.CMIP.EC-Earth-Consortium.EC-Earth3-CC.piControl.r1i1p1f1.Amon.tas.gr.v20210330|esgf.bsc.es downloading


True

In [8]:
fh.loadModelData('GISS-E2-1-G', 'ts_Amon', 'piControl', 'r1i1p1f2')

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 90, lon: 144, time: 4140)
Coordinates:
  * time       (time) object 7550-01-16 00:00:00 ... 7894-12-16 00:00:00
  * lat        (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon        (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(612, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(612, 90, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(612, 144, 2), meta=np.ndarray>
    ts         (time, lat, lon) float32 dask.array<chunksize=(612, 90, 144), meta=np.ndarray>
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                Kenneth Lo (cdkkl@giss.nasa.gov)
    ...                     ...
    title:                  GISS-E2-1-G output prepared for CMIP6
    tracking_id:            hdl:21.14100/7c16d62e-003f-46ca-9479-f4c6e596dda0
    variable_id:            ts
    variant_label:          r1i1p1f2
    license:                CMIP6 model data produced by NASA Goddard Institu...
    cmor_version:           3.3.2

In [26]:
fh.loadModelData('NorESM2-LM', 'tas_Amon', 'ssp585', 'r1i1p1f1')

CMIP6.ScenarioMIP.NCC.NorESM2-LM.ssp585.r1i1p1f1.Amon.tas.gn.v20191108|esgf-data3.diasjp.net downloading
CMIP6.ScenarioMIP.NCC.NorESM2-LM.ssp585.r1i1p1f1.Amon.tas.gn.v20191108|esgf-data1.llnl.gov downloading
CMIP6.ScenarioMIP.NCC.NorESM2-LM.ssp585.r1i1p1f1.Amon.tas.gn.v20191108|esgf3.dkrz.de downloading
CMIP6.ScenarioMIP.NCC.NorESM2-LM.ssp585.r1i1p1f1.Amon.tas.gn.v20191108|noresg.nird.sigma2.no downloading
CMIP6.ScenarioMIP.NCC.NorESM2-LM.ssp585.r1i1p1f1.Amon.tas.gn.v20191108|esgf.nci.org.au downloading
CMIP6.ScenarioMIP.NCC.NorESM2-LM.ssp585.r1i1p1f1.Amon.tas.gn.v20191108|esgf-data3.ceda.ac.uk downloading


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lon: 144, time: 1032)
Coordinates:
  * time       (time) object 2015-01-16 00:00:00 ... 2100-12-16 00:00:00
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(72, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(72, 96, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(72, 144, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(72, 96, 144), meta=np.ndarray>
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               ScenarioMIP
    branch_method:             Hybrid-restart from year 2015-01-01 of historical
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     60225.0
    ...                        ...
    title:                     NorESM2-LM output prepared for CMIP6
    variable_id:               tas
    variant_label:             r1i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/67c1b06a-9dd3-4075-882c-47c159724b38

# Prototyping for reference / sandbox

In [9]:
from pyesgf.search import SearchConnection

conn = SearchConnection('https://esgf-node.llnl.gov/esg-search')
#conn = SearchConnection('https://esgf-data.dkrz.de/esg-search')
#conn = SearchConnection('https://esgf.nci.org.au/esg-search')

ctx = conn.new_context(
    project='CMIP6', 
    source_id='EC-Earth3-CC', 
    experiment_id='ssp585', 
    variable='ts_Amon'.split('_')[0], 
    frequency='mon', 
    variant_label='r1i1p1f1',
    data_node='esgf-data1.llnl.gov'
)
ctx.hit_count

1

In [10]:
results = ctx.search()
[result.dataset_id for result in results]

['CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-CC.ssp585.r1i1p1f1.Amon.ts.gr.v20210113|esgf-data1.llnl.gov']

In [11]:
len(results)

1

In [12]:
results[0].urls

KeyError: 'url'

In [ ]:
%%writefile dl.sh
results[0].file_context().get_download_script()

In [ ]:
import subprocess
subprocess.check_output('bash dl.sh')

In [ ]:
from pyesgf.logon import LogonManager
lm = LogonManager()
lm.logoff()
lm.is_logged_on()

In [ ]:
myproxy_host = 'esgf-data.dkrz.de'
lm.logon(username=None, password=None, hostname=myproxy_host)
lm.is_logged_on()